# Calculate vertical temperature gradient between surface and pressure levels 925, 850, 700
RUFMOD ONLY

* **Description**: Reads in and creates seasonal and ensemble means and vertically interpolates
* **Input data**: Rufmod output in timeseries format
* **Output data**: Netcdf file with output
* **Creator**: Alice DuVivier
* **Date**: April 2022

The rufmod experiments were performed where the sea ice roughness over Arctic sea ice regions was set to be equal to what it would be over open ocean. This is to better understand ice-atmosphere coupling, processes, and feedbacks.

In [1]:
import xarray as xr
import numpy as np
from datetime import timedelta
import glob

import pop_tools

import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib.gridspec import GridSpec

import geocat.datafiles as gdf
import geocat.viz.util as gvutil
from geocat.viz import cmaps as gvcmaps
import geocat.comp as gcomp

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.stats import linregress,pearsonr, t

import dask
import intake
from distributed import Client
from ncar_jobqueue import NCARCluster

/glade/work/duvivier/miniconda3/envs/analysis3/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
# spin up dask cluster

import dask

# Use dask jobqueue
from dask_jobqueue import PBSCluster

# Import a client
from dask.distributed import Client

# Setup your PBSCluster
cluster = PBSCluster(
    cores=36, # The number of cores you want
    memory='300 GB', # Amount of memory
    processes=9, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus=36:mem=300GB', # Specify resources
    project='P93300665', # Input your project ID here
    walltime='00:30:00', # Amount of wall time
    interface='ib0', # Interface to use
)
# Scale up
cluster.scale(jobs=4)

# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

In [3]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/duvivier/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/duvivier/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.54:42169,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/duvivier/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Manually set variables

In [4]:
# list the variables to load
var_in_1 = 'T'
var_in_2 = 'TS'

## Load rufmod experiments

In [5]:
# Load "rufmod" data
#choose cases and data paths
case1 = 'b.e21.BSSP370.f09_g17.rufmod.001'
case2 = 'b.e21.BSSP370.f09_g17.rufmod.002'
case3 = 'b.e21.BSSP370.f09_g17.rufmod.003'
case4 = 'b.e21.BSSP370.f09_g17.rufmod.004'
case5 = 'b.e21.BSSP370.f09_g17.rufmod.005'

# set base directory where all data live
data_dir = '/glade/campaign/cesm/development/pcwg/projects/arctic_cyclones/rufmod_expts/'
# set individual data directories
data_dir1 = data_dir+case1+'/atm/proc/tseries/month_1/'
data_dir2 = data_dir+case2+'/atm/proc/tseries/month_1/'
data_dir3 = data_dir+case3+'/atm/proc/tseries/month_1/'
data_dir4 = data_dir+case4+'/atm/proc/tseries/month_1/'
data_dir5 = data_dir+case5+'/atm/proc/tseries/month_1/'

In [6]:
%%time
#reading in files
print("loading "+var_in_1)   
ds1_1 = []
ds2_1 = []
ds3_1 = []
ds4_1 = []
ds5_1 = []
my_files=sorted(glob.glob(data_dir1+case1+'.cam.h0.'+var_in_1+'.*.nc'))
ds1_1=xr.open_mfdataset(my_files,combine='by_coords',chunks={'time':129}, parallel=True, compat='override', coords='minimal')
my_files=sorted(glob.glob(data_dir2+case2+'.cam.h0.'+var_in_1+'.*.nc'))
ds2_1=xr.open_mfdataset(my_files,combine='by_coords',chunks={'time':129}, parallel=True, compat='override', coords='minimal')
my_files=sorted(glob.glob(data_dir3+case3+'.cam.h0.'+var_in_1+'.*.nc'))
ds3_1=xr.open_mfdataset(my_files,combine='by_coords',chunks={'time':129}, parallel=True, compat='override', coords='minimal')    
my_files=sorted(glob.glob(data_dir4+case4+'.cam.h0.'+var_in_1+'.*.nc'))
ds4_1=xr.open_mfdataset(my_files,combine='by_coords',chunks={'time':129}, parallel=True, compat='override', coords='minimal')
my_files=sorted(glob.glob(data_dir5+case5+'.cam.h0.'+var_in_1+'.*.nc'))
ds5_1=xr.open_mfdataset(my_files,combine='by_coords',chunks={'time':129}, parallel=True, compat='override', coords='minimal')

print("loading "+var_in_2)   
ds1_2 = []
ds2_2 = []
ds3_2 = []
ds4_2 = []
ds5_2 = []
my_files=sorted(glob.glob(data_dir1+case1+'.cam.h0.'+var_in_2+'.*.nc'))
ds1_2=xr.open_mfdataset(my_files,combine='by_coords',chunks={'time':129}, parallel=True, compat='override', coords='minimal')
my_files=sorted(glob.glob(data_dir2+case2+'.cam.h0.'+var_in_2+'.*.nc'))
ds2_2=xr.open_mfdataset(my_files,combine='by_coords',chunks={'time':129}, parallel=True, compat='override', coords='minimal')
my_files=sorted(glob.glob(data_dir3+case3+'.cam.h0.'+var_in_2+'.*.nc'))
ds3_2=xr.open_mfdataset(my_files,combine='by_coords',chunks={'time':129}, parallel=True, compat='override', coords='minimal')    
my_files=sorted(glob.glob(data_dir4+case4+'.cam.h0.'+var_in_2+'.*.nc'))
ds4_2=xr.open_mfdataset(my_files,combine='by_coords',chunks={'time':129}, parallel=True, compat='override', coords='minimal')
my_files=sorted(glob.glob(data_dir5+case5+'.cam.h0.'+var_in_2+'.*.nc'))
ds5_2=xr.open_mfdataset(my_files,combine='by_coords',chunks={'time':129}, parallel=True, compat='override', coords='minimal')

loading T
loading TS
CPU times: user 1.26 s, sys: 351 ms, total: 1.61 s
Wall time: 18.8 s


In [7]:
# concatenate them into a single array
futures_1 = xr.concat([ds1_1,ds2_1,ds3_1,ds4_1,ds5_1],dim='member_id')
futures_2 = xr.concat([ds1_2,ds2_2,ds3_2,ds4_2,ds5_2],dim='member_id')

In [8]:
futures_1.T

<xarray.DataArray 'T' (member_id: 5, time: 1032, lev: 32, lat: 192, lon: 288)>
dask.array<concatenate, shape=(5, 1032, 32, 192, 288), dtype=float32, chunksize=(1, 129, 32, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * time     (time) object 2015-02-01 00:00:00 ... 2101-01-01 00:00:00
Dimensions without coordinates: member_id
Attributes:
    mdims:         1
    units:         K
    long_name:     Temperature
    cell_methods:  time: mean

In [9]:
# set member_id values
futures_1.member_id.values
futures_2.member_id.values

# assign member_id as coordinate array
futures_1 = futures_1.assign_coords({"member_id": futures_1.member_id.values})
futures_2 = futures_2.assign_coords({"member_id": futures_2.member_id.values})

In [10]:
# Shift months by one to be center of time period.
# Take average of the time bounds to get middle of month
# will lose some attributes with time, so may need to put this back in later...
futures_1['time'] = futures_1.time_bnds.load().mean(dim='nbnd').sel(member_id=0)
futures_2['time'] = futures_2.time_bnds.load().mean(dim='nbnd').sel(member_id=0)

In [11]:
# get just NH slice
futures_1_masked = futures_1.isel(lat=slice(164,192))
futures_2_masked = futures_2.isel(lat=slice(164,192))

In [12]:
# grab variables of interest
T_rufmod = futures_1_masked[var_in_1]
TS_rufmod = futures_2_masked[var_in_2]

## Calculate pressure on regular levels

### Get surface pressure

In [13]:
# We will need surface pressure and reference pressure for interpolation

# set a surface reference pressure [Pa]
p0 = 100000 

# set variable name
var_in_3 = 'PS'

In [14]:
# Load surface pressure from rufmod
print("loading rufmod "+var_in_3)   
ds1_3 = []
ds2_3 = []
ds3_3 = []
ds4_3 = []
ds5_3 = []
my_files=sorted(glob.glob(data_dir1+case1+'.cam.h0.'+var_in_3+'.*.nc'))
ds1_3=xr.open_mfdataset(my_files,combine='by_coords',chunks={}, parallel=True, compat='override', coords='minimal')
my_files=sorted(glob.glob(data_dir2+case2+'.cam.h0.'+var_in_3+'.*.nc'))
ds2_3=xr.open_mfdataset(my_files,combine='by_coords',chunks={}, parallel=True, compat='override', coords='minimal')
my_files=sorted(glob.glob(data_dir3+case3+'.cam.h0.'+var_in_3+'.*.nc'))
ds3_3=xr.open_mfdataset(my_files,combine='by_coords',chunks={}, parallel=True, compat='override', coords='minimal')    
my_files=sorted(glob.glob(data_dir4+case4+'.cam.h0.'+var_in_3+'.*.nc'))
ds4_3=xr.open_mfdataset(my_files,combine='by_coords',chunks={}, parallel=True, compat='override', coords='minimal')
my_files=sorted(glob.glob(data_dir5+case5+'.cam.h0.'+var_in_3+'.*.nc'))
ds5_3=xr.open_mfdataset(my_files,combine='by_coords',chunks={}, parallel=True, compat='override', coords='minimal')

loading rufmod PS


In [15]:
# combine the datasets
futures_3 = xr.concat([ds1_3,ds2_3,ds3_3,ds4_3,ds5_3],dim='member_id')

# set member_id values
futures_3.member_id.values

# assign member_id as coordinate array
futures_3.assign_coords({"member_id": futures_3.member_id.values})

<xarray.Dataset>
Dimensions:       (lat: 192, zlon: 1, member_id: 5, time: 1032, nbnd: 2, lon: 288, lev: 32, ilev: 33)
Coordinates:
  * lat           (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * zlon          (zlon) float64 0.0
  * lon           (lon) float64 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lev           (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev          (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * time          (time) object 2015-02-01 00:00:00 ... 2101-01-01 00:00:00
  * member_id     (member_id) int64 0 1 2 3 4
Dimensions without coordinates: nbnd
Data variables: (12/27)
    zlon_bnds     (member_id, time, zlon, nbnd) float64 dask.array<chunksize=(1, 600, 1, 2), meta=np.ndarray>
    gw            (member_id, time, lat) float64 dask.array<chunksize=(1, 600, 192), meta=np.ndarray>
    hyam          (member_id, time, lev) float64 dask.array<chunksize=(1, 600, 32), meta=np.ndarray>
    hybm          (member_id, time, lev) float64 dask.array<chunksize=(1, 600, 32), meta=np.ndarray>
    P0            (member_id, time) float64 1e+05 1e+05 1e+05 ... 1e+05 1e+05
    hyai          (member_id, time, ilev) float64 dask.array<chunksize=(1, 600, 33), meta=np.ndarray>
    ...            ...
    n2ovmr        (member_id, time) float64 dask.array<chunksize=(1, 600), meta=np.ndarray>
    f11vmr        (member_id, time) float64 dask.array<chunksize=(1, 600), meta=np.ndarray>
    f12vmr        (member_id, time) float64 dask.array<chunksize=(1, 600), meta=np.ndarray>
    sol_tsi       (member_id, time) float64 dask.array<chunksize=(1, 600), meta=np.ndarray>
    nsteph        (member_id, time) float64 dask.array<chunksize=(1, 600), meta=np.ndarray>
    PS            (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 600, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BSSP370.f09_g17.rufmod.001
    logname:           duvivier
    host:              cheyenne3
    initial_file:      b.e21.BHIST.f09_g17.CMIP6-historical.rufmod.cam.i.2015...
    topography_file:   /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/fv_0.9x1...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  month_1

In [16]:
# Shift months by one to be center of time period.
# Take average of the time bounds to get middle of month
# will lose some attributes with time, so may need to put this back in later...
futures_3['time'] = futures_3.time_bnds.load().mean(dim='nbnd').sel(member_id=0)

# get just NH slice
futures_3_masked = futures_3.isel(lat=slice(164,192))

# grab variable of interest
PS_rufmod = futures_3_masked[var_in_3]

### Load the vertical parameters

In [17]:
# grab parameters for interpolation
hyam_rufmod = futures_1_masked["hyam"]
hybm_rufmod = futures_1_masked["hybm"]

### Interpolate to new pressure levels from hybrid levels

In [18]:
# Specify output pressure levels
new_levels = np.array([925, 850, 700, 500])  # in millibars
new_levels = new_levels * 100  # convert to Pascals

In [19]:
%%time
T_rufmod_interp = gcomp.interp_hybrid_to_pressure(T_rufmod,
                                                   PS_rufmod,
                                                   hyam_rufmod, hybm_rufmod, p0 = p0,
                                                   new_levels=new_levels,
                                                   method='linear')

CPU times: user 15.1 ms, sys: 0 ns, total: 15.1 ms
Wall time: 15.1 ms


In [20]:
T_rufmod_interp

<xarray.DataArray 'T' (member_id: 5, time: 1032, plev: 4, lat: 28, lon: 288)>
dask.array<_vertical_remap, shape=(5, 1032, 4, 28, 288), dtype=float32, chunksize=(1, 129, 4, 28, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float64 64.55 65.5 66.44 67.38 ... 87.17 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * plev       (plev) int64 92500 85000 70000 50000
  * time       (time) object 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
  * member_id  (member_id) int64 0 1 2 3 4
Attributes:
    mdims:         1
    units:         K
    long_name:     Temperature
    cell_methods:  time: mean

# Actually load data

In [21]:
T_rufmod_interp.load()
TS_rufmod.load()

#T_rufmod_interp.persist()
#TS_rufmod.persist()

<xarray.DataArray 'TS' (member_id: 5, time: 1032, lat: 28, lon: 288)>
array([[[[281.05746, 281.11447, 281.2591 , ..., 281.22598, 281.12366,
          281.05792],
         [280.45514, 280.56323, 280.7759 , ..., 280.38464, 280.3714 ,
          280.38895],
         [279.78674, 279.96628, 280.21915, ..., 279.4418 , 279.5316 ,
          279.65604],
         ...,
         [245.84915, 245.88068, 245.91075, ..., 245.7521 , 245.78387,
          245.8157 ],
         [245.97737, 245.98969, 246.00276, ..., 245.93716, 245.95193,
          245.96494],
         [246.11044, 246.11055, 246.11064, ..., 246.1101 , 246.11023,
          246.11034]],

        [[280.88293, 280.97717, 281.0798 , ..., 280.79642, 280.80057,
          280.81738],
         [280.32556, 280.53314, 280.734  , ..., 279.93695, 280.04108,
          280.14954],
         [279.7083 , 279.95776, 280.1756 , ..., 279.0265 , 279.22662,
          279.45627],
...
         [272.3739 , 272.3752 , 272.3766 , ..., 272.37097, 272.37186,
          272.3728 ],
         [272.39603, 272.39633, 272.3967 , ..., 272.3952 , 272.39545,
          272.39572],
         [272.40793, 272.40796, 272.40796, ..., 272.40793, 272.40793,
          272.40793]],

        [[280.84473, 281.10858, 281.33197, ..., 279.79388, 280.15424,
          280.52048],
         [280.59805, 280.87585, 281.11084, ..., 279.55795, 279.92926,
          280.28128],
         [280.3302 , 280.59912, 280.82535, ..., 279.3669 , 279.7161 ,
          280.0435 ],
         ...,
         [271.77924, 271.78427, 271.78915, ..., 271.76492, 271.7697 ,
          271.77423],
         [271.79318, 271.79486, 271.79663, ..., 271.7874 , 271.7896 ,
          271.79147],
         [271.7943 , 271.79453, 271.79474, ..., 271.79352, 271.7938 ,
          271.79407]]]], dtype=float32)
Coordinates:
  * lat        (lat) float64 64.55 65.5 66.44 67.38 ... 87.17 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time       (time) object 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
  * member_id  (member_id) int64 0 1 2 3 4
Attributes:
    units:         K
    long_name:     Surface temperature (radiative)
    cell_methods:  time: mean

## Now calculate temperature gradients that are relevant (based on wind speed aloft trends)

- TS with T925, T850, T700

In [22]:
Tgrad = T_rufmod_interp - TS_rufmod

In [23]:
Tgrad.persist()

<xarray.DataArray (member_id: 5, time: 1032, plev: 4, lat: 28, lon: 288)>
array([[[[[-8.13861338e+00, -8.15602786e+00, -8.24905795e+00, ...,
           -8.59738203e+00, -8.37520888e+00, -8.20882344e+00],
          [-8.02554727e+00, -8.02559172e+00, -8.17269978e+00, ...,
           -8.44822012e+00, -8.24571153e+00, -8.11124511e+00],
          [-7.93590082e+00, -7.95299330e+00, -8.10012300e+00, ...,
           -8.28933031e+00, -8.12007145e+00, -8.00635004e+00],
          ...,
          [ 4.56012647e+00,  4.53269799e+00,  4.50543744e+00, ...,
            4.63704305e+00,  4.61282130e+00,  4.58770987e+00],
          [ 4.55845608e+00,  4.54134589e+00,  4.52117995e+00, ...,
            4.60624091e+00,  4.59121070e+00,  4.57520346e+00],
          [ 4.63573026e+00,  4.63563080e+00,  4.63553925e+00, ...,
            4.63606596e+00,  4.63594389e+00,  4.63583707e+00]],

         [[-1.35884609e+01, -1.35780651e+01, -1.36717433e+01, ...,
           -1.42178797e+01, -1.39539980e+01, -1.37133937e+01],
          [-1.34721320e+01, -1.34263463e+01, -1.35257555e+01, ...,
           -1.39604646e+01, -1.37579933e+01, -1.36135356e+01],
          [-1.33000148e+01, -1.32663005e+01, -1.34126769e+01, ...,
           -1.36396512e+01, -1.34706618e+01, -1.33843784e+01],
...
          [-1.96223826e+01, -1.96309042e+01, -1.96355434e+01, ...,
           -1.95960773e+01, -1.96038152e+01, -1.96129156e+01],
          [-2.01684763e+01, -2.01658767e+01, -2.01656190e+01, ...,
           -2.01796974e+01, -2.01745408e+01, -2.01712963e+01],
          [-2.04845337e+01, -2.04847465e+01, -2.04849594e+01, ...,
           -2.04837532e+01, -2.04840149e+01, -2.04842888e+01]],

         [[-3.30907232e+01, -3.33827701e+01, -3.36282242e+01, ...,
           -3.19472369e+01, -3.23151185e+01, -3.27266498e+01],
          [-3.29322729e+01, -3.32212204e+01, -3.34518529e+01, ...,
           -3.19420074e+01, -3.22613853e+01, -3.26043247e+01],
          [-3.27371005e+01, -3.30003685e+01, -3.32296737e+01, ...,
           -3.19056005e+01, -3.21687021e+01, -3.24586185e+01],
          ...,
          [-3.36475075e+01, -3.36469986e+01, -3.36468301e+01, ...,
           -3.36528252e+01, -3.36505311e+01, -3.36485889e+01],
          [-3.41044808e+01, -3.41024124e+01, -3.41007509e+01, ...,
           -3.41115254e+01, -3.41092625e+01, -3.41067934e+01],
          [-3.44675852e+01, -3.44677988e+01, -3.44680125e+01, ...,
           -3.44667917e+01, -3.44670664e+01, -3.44673411e+01]]]]])
Coordinates:
  * lat        (lat) float64 64.55 65.5 66.44 67.38 ... 87.17 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * plev       (plev) int64 92500 85000 70000 50000
  * time       (time) object 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
  * member_id  (member_id) int64 0 1 2 3 4
Attributes:
    mdims:         1
    units:         K
    long_name:     Temperature
    cell_methods:  time: mean

## Calculate Seasonal Means

In [24]:
season_names = ['OND','JFM', 'AMJ', 'JAS']

In [25]:
# find total years
xarr_rufmod = TS_rufmod.coords['time.year'][(TS_rufmod.coords['time.month']==1)]

In [29]:
# Loop through seasons - rufmod

# make numpy array to fill and specify dimensions we want
seas_array_rufmod_1 = np.zeros([len(season_names),len(xarr_rufmod),len(Tgrad.member_id),len(Tgrad.plev),len(Tgrad.lat),len(Tgrad.lon)])
seas_array_rufmod_2 = np.zeros([len(season_names),len(xarr_rufmod),len(Tgrad.member_id),len(Tgrad.plev),len(Tgrad.lat),len(Tgrad.lon)])
seas_array_rufmod_3 = np.zeros([len(season_names),len(xarr_rufmod),len(Tgrad.member_id),len(Tgrad.lat),len(Tgrad.lon)])

for s_count, ss in enumerate(season_names):
    print(ss)
    ###TGRAD
    # get temporary array of just these month by season
    if ss == 'JFM':
        temp1 = Tgrad.isel(time=Tgrad.time.dt.month.isin([1,2,3]))
    if ss == 'AMJ':
        temp1 = Tgrad.isel(time=Tgrad.time.dt.month.isin([4,5,6]))
    if ss == 'JAS':
        temp1 = Tgrad.isel(time=Tgrad.time.dt.month.isin([7,8,9]))
    if ss == 'OND':
        temp1 = Tgrad.isel(time=Tgrad.time.dt.month.isin([10,11,12]))
    # now loop through years to get the seasonal average by year for each ensemble member
    for y_count, yy in enumerate(xarr_rufmod):
        # select only the indexes for this year
        temp1a = temp1.isel(time=temp1.time.dt.year.isin([yy])).mean(dim='time')
        seas_array_rufmod_1[s_count,y_count,:,:,:,:] = temp1a   

    ###TPLEV
    # get temporary array of just these month by season
    if ss == 'JFM':
        temp1 = T_rufmod_interp.isel(time=T_rufmod_interp.time.dt.month.isin([1,2,3]))
    if ss == 'AMJ':
        temp1 = T_rufmod_interp.isel(time=T_rufmod_interp.time.dt.month.isin([4,5,6]))
    if ss == 'JAS':
        temp1 = T_rufmod_interp.isel(time=T_rufmod_interp.time.dt.month.isin([7,8,9]))
    if ss == 'OND':
        temp1 = T_rufmod_interp.isel(time=T_rufmod_interp.time.dt.month.isin([10,11,12]))
    # now loop through years to get the seasonal average by year for each ensemble member
    for y_count, yy in enumerate(xarr_rufmod):
        # select only the indexes for this year
        temp1a = temp1.isel(time=temp1.time.dt.year.isin([yy])).mean(dim='time')
        seas_array_rufmod_2[s_count,y_count,:,:,:,:] = temp1a 
        
    ###TS
    # get temporary array of just these month by season
    if ss == 'JFM':
        temp1 = TS_rufmod.isel(time=TS_rufmod.time.dt.month.isin([1,2,3]))
    if ss == 'AMJ':
        temp1 = TS_rufmod.isel(time=TS_rufmod.time.dt.month.isin([4,5,6]))
    if ss == 'JAS':
        temp1 = TS_rufmod.isel(time=TS_rufmod.time.dt.month.isin([7,8,9]))
    if ss == 'OND':
        temp1 = TS_rufmod.isel(time=TS_rufmod.time.dt.month.isin([10,11,12]))
    # now loop through years to get the seasonal average by year for each ensemble member
    for y_count, yy in enumerate(xarr_rufmod):
        # select only the indexes for this year
        temp1a = temp1.isel(time=temp1.time.dt.year.isin([yy])).mean(dim='time')
        seas_array_rufmod_3[s_count,y_count,:,:,:] = temp1a 
        

OND
JFM
AMJ
JAS


In [31]:
print(seas_array_rufmod_1.shape)
print(seas_array_rufmod_2.shape)
print(seas_array_rufmod_3.shape)

(4, 86, 5, 4, 28, 288)
(4, 86, 5, 4, 28, 288)
(4, 86, 5, 28, 288)


In [37]:
# convert the numpy array to a xarray for easier plotting
TGRAD_seas_rufmod = xr.DataArray(seas_array_rufmod_1,dims=('season','time','member_id','plev','lat','lon'))
T_seas_rufmod = xr.DataArray(seas_array_rufmod_2,dims=('season','time','member_id','plev','lat','lon'))
TS_seas_rufmod = xr.DataArray(seas_array_rufmod_3,dims=('season','time','member_id','lat','lon'))

In [48]:
# set coordinate arrays
TGRAD_seas_rufmod['season'] = season_names
TGRAD_seas_rufmod['time'] = xarr_rufmod
TGRAD_seas_rufmod['member_id'] = Tgrad['member_id'].data
TGRAD_seas_rufmod['plev'] = Tgrad['plev'].values
TGRAD_seas_rufmod['lat'] = Tgrad['lat'].values
TGRAD_seas_rufmod['lon'] = Tgrad['lon'].values

T_seas_rufmod['season'] = season_names
T_seas_rufmod['time'] = xarr_rufmod
T_seas_rufmod['member_id'] = Tgrad['member_id']
T_seas_rufmod['plev'] = Tgrad['plev'].values
T_seas_rufmod['lat'] = Tgrad['lat'].values
T_seas_rufmod['lon'] = Tgrad['lon'].values

TS_seas_rufmod['season'] = season_names
TS_seas_rufmod['time'] = xarr_rufmod
TS_seas_rufmod['member_id'] = Tgrad['member_id']
TS_seas_rufmod['lat'] = Tgrad['lat'].values
TS_seas_rufmod['lon'] = Tgrad['lon'].values

## Write out files

In [ ]:
# quick and dirty way to save a file!

# save rufmod expt, rename the variable so it makes sense
#fout = 'rufmod_vertical_seas_ens_mean_WS'
#
#WS_seas_ens_mean_rufmod.to_dataset(name='vert_ws').to_netcdf(fout+'.nc')

### TGRAD

In [49]:
#set info to write out
out_tag = 'TGRAD'
units = 'K'
longname = 'temperature gradient (Taloft - Tsfc)'

fout = 'rufmod_seas_'+out_tag

In [50]:
ds_to_save = TGRAD_seas_rufmod

In [51]:
# check how big this will be to write out in GB
ds_to_save.nbytes/(1024**3)

0.413360595703125

In [52]:
# assign some attributes
refdata = {'Author': 'Alice DuVivier', 'units':units, 'longname':longname}

ds_to_save.attrs = refdata

In [53]:
# check data
ds_to_save


<xarray.DataArray (season: 4, time: 86, member_id: 5, plev: 4, lat: 28, lon: 288)>
array([[[[[[-7.43804576e+00, -7.56962300e+00, -7.80732341e+00, ...,
            -7.54284313e+00, -7.45526379e+00, -7.41857781e+00],
           [-7.44948225e+00, -7.51885439e+00, -7.70111116e+00, ...,
            -7.47582709e+00, -7.45423787e+00, -7.44739593e+00],
           [-7.53551668e+00, -7.60358392e+00, -7.73442780e+00, ...,
            -7.38710421e+00, -7.45455789e+00, -7.51867934e+00],
           ...,
           [ 2.92186593e+00,  2.89436437e+00,  2.87080599e+00, ...,
             3.00187724e+00,  2.97655837e+00,  2.95200995e+00],
           [ 2.82749976e+00,  2.82382829e+00,  2.81944229e+00, ...,
             2.84315750e+00,  2.83569109e+00,  2.83115328e+00],
           [ 2.65221924e+00,  2.65152242e+00,  2.65088411e+00, ...,
             2.65476492e+00,  2.65382905e+00,  2.65299235e+00]],

          [[-1.18453954e+01, -1.20261439e+01, -1.22846851e+01, ...,
            -1.20108410e+01, -1.18708971e+01, -1.17851961e+01],
           [-1.19079898e+01, -1.20057614e+01, -1.22196218e+01, ...,
            -1.18585007e+01, -1.18415703e+01, -1.18425774e+01],
           [-1.19676188e+01, -1.20591316e+01, -1.22118095e+01, ...,
            -1.17135117e+01, -1.17833738e+01, -1.18870159e+01],
...
           [-5.42822584e+00, -5.43040257e+00, -5.43199903e+00, ...,
            -5.42185241e+00, -5.42339977e+00, -5.42553082e+00],
           [-5.49690055e+00, -5.49637813e+00, -5.49582656e+00, ...,
            -5.49961607e+00, -5.49831817e+00, -5.49747281e+00],
           [-5.57929042e+00, -5.57925926e+00, -5.57913880e+00, ...,
            -5.57948905e+00, -5.57939156e+00, -5.57936059e+00]],

          [[-2.66933798e+01, -2.70158491e+01, -2.73126036e+01, ...,
            -2.57280522e+01, -2.60165597e+01, -2.63485194e+01],
           [-2.67489624e+01, -2.70381250e+01, -2.73085675e+01, ...,
            -2.58168785e+01, -2.61269508e+01, -2.64395185e+01],
           [-2.67259644e+01, -2.69883741e+01, -2.72084021e+01, ...,
            -2.58527589e+01, -2.61479408e+01, -2.64467969e+01],
           ...,
           [-1.93880903e+01, -1.93886357e+01, -1.93889870e+01, ...,
            -1.93882020e+01, -1.93879141e+01, -1.93877731e+01],
           [-1.94787507e+01, -1.94775761e+01, -1.94765910e+01, ...,
            -1.94827109e+01, -1.94812826e+01, -1.94798852e+01],
           [-1.96239334e+01, -1.96238858e+01, -1.96238266e+01, ...,
            -1.96241267e+01, -1.96240486e+01, -1.96239945e+01]]]]]])
Coordinates:
  * season     (season) <U3 'OND' 'JFM' 'AMJ' 'JAS'
  * time       (time) int64 2015 2016 2017 2018 2019 ... 2097 2098 2099 2100
  * member_id  (member_id) int64 0 1 2 3 4
  * plev       (plev) int64 92500 85000 70000 50000
  * lat        (lat) float64 64.55 65.5 66.44 67.38 ... 87.17 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    Author:    Alice DuVivier
    units:     K
    longname:  temperature gradient (Taloft - Tsfc)

In [54]:
ds_to_save.to_netcdf(fout+'.nc')  # how to save file

### T @ PLEVS

In [55]:
#set info to write out
out_tag = 'T'
units = 'K'
longname = 'temperature aloft at given plevels'

fout = 'rufmod_seas_'+out_tag

In [56]:
ds_to_save = T_seas_rufmod

In [57]:
# check how big this will be to write out in GB
ds_to_save.nbytes/(1024**3)

0.413360595703125

In [58]:
# assign some attributes
refdata = {'Author': 'Alice DuVivier', 'units':units, 'longname':longname}

ds_to_save.attrs = refdata

In [59]:
# check data
ds_to_save


<xarray.DataArray (season: 4, time: 86, member_id: 5, plev: 4, lat: 28, lon: 288)>
array([[[[[[274.74991201, 274.81156189, 274.88410318, ...,
            274.57252349, 274.61986031, 274.66979092],
           [274.1318471 , 274.25320982, 274.37102223, ...,
            273.81400852, 273.90817872, 274.00481354],
           [273.41863574, 273.5880563 , 273.75813608, ...,
            273.00865792, 273.12248883, 273.24913276],
           ...,
           [257.79949553, 257.80570572, 257.81196301, ...,
            257.77692709, 257.78509618, 257.79267767],
           [257.62387122, 257.62710181, 257.62991795, ...,
            257.61248021, 257.61653928, 257.62031242],
           [257.43908448, 257.43908448, 257.43907686, ...,
            257.43907686, 257.43907686, 257.43908448]],

          [[270.34256236, 270.35504096, 270.4067415 , ...,
            270.10452558, 270.20422701, 270.30317264],
           [269.67333955, 269.76630278, 269.8525116 , ...,
            269.43133495, 269.52084628, 269.6096321 ],
           [268.98653359, 269.13250859, 269.28075443, ...,
            268.68225046, 268.79367291, 268.88079617],
...
           [268.32776578, 268.3266063 , 268.32581347, ...,
            268.33231833, 268.33139149, 268.32970803],
           [268.19899178, 268.19963627, 268.20033026, ...,
            268.19555401, 268.19716726, 268.19824659],
           [268.10099725, 268.10098773, 268.10105733, ...,
            268.1009919 , 268.10101819, 268.10098813]],

          [[257.83948766, 257.80307993, 257.76249002, ...,
            257.92236898, 257.9034029 , 257.87590684],
           [257.51698814, 257.49198568, 257.4632424 , ...,
            257.59203674, 257.56949083, 257.54535494],
           [257.23650918, 257.21836215, 257.20519242, ...,
            257.29794704, 257.2842797 , 257.26257408],
           ...,
           [254.36790133, 254.36837321, 254.3688255 , ...,
            254.36596869, 254.36687715, 254.36746575],
           [254.2171416 , 254.2184383 , 254.21956582, ...,
            254.21245919, 254.21420283, 254.21583423],
           [254.05635426, 254.05636114, 254.05636956, ...,
            254.05635426, 254.05636114, 254.05635426]]]]]])
Coordinates:
  * season     (season) <U3 'OND' 'JFM' 'AMJ' 'JAS'
  * time       (time) int64 2015 2016 2017 2018 2019 ... 2097 2098 2099 2100
  * member_id  (member_id) int64 0 1 2 3 4
  * plev       (plev) int64 92500 85000 70000 50000
  * lat        (lat) float64 64.55 65.5 66.44 67.38 ... 87.17 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    Author:    Alice DuVivier
    units:     K
    longname:  temperature aloft at given plevels

In [60]:
ds_to_save.to_netcdf(fout+'.nc')  # how to save file

### TSFC

In [61]:
#set info to write out
out_tag = 'TS'
units = 'K'
longname = 'surface temperature'

fout = 'rufmod_seas_'+out_tag

In [62]:
ds_to_save = TS_seas_rufmod

In [63]:
# check how big this will be to write out in GB
ds_to_save.nbytes/(1024**3)

0.10334014892578125

In [64]:
# assign some attributes
refdata = {'Author': 'Alice DuVivier', 'units':units, 'longname':longname}

ds_to_save.attrs = refdata

In [65]:
# check data
ds_to_save


<xarray.DataArray (season: 4, time: 86, member_id: 5, lat: 28, lon: 288)>
array([[[[[282.18795776, 282.38119507, 282.69143677, ...,
           282.11538696, 282.07510376, 282.08834839],
          [281.58132935, 281.77206421, 282.07214355, ...,
           281.28982544, 281.36242676, 281.45223999],
          [280.95413208, 281.19165039, 281.49255371, ...,
           280.39575195, 280.57705688, 280.76779175],
          ...,
          [254.87762451, 254.91133118, 254.94114685, ...,
           254.77503967, 254.80853271, 254.84065247],
          [254.79638672, 254.80326843, 254.81047058, ...,
           254.76933289, 254.78083801, 254.78916931],
          [254.78686523, 254.78755188, 254.78819275, ...,
           254.78430176, 254.78523254, 254.78608704]],

         [[282.32598877, 282.56228638, 282.93301392, ...,
           282.28372192, 282.21725464, 282.21203613],
          [281.7071228 , 281.95700073, 282.33480835, ...,
           281.30627441, 281.41329956, 281.5340271 ],
          [281.03479004, 281.33563232, 281.71688843, ...,
           280.26928711, 280.52792358, 280.78942871],
...
          [274.37527466, 274.37564087, 274.37588501, ...,
           274.37426758, 274.37460327, 274.37490845],
          [274.37646484, 274.37588501, 274.37527466, ...,
           274.37808228, 274.37759399, 274.37704468],
          [274.37805176, 274.37802124, 274.37796021, ...,
           274.37820435, 274.37817383, 274.37808228]],

         [[284.53286743, 284.81893921, 285.07510376, ...,
           283.65042114, 283.91995239, 284.22445679],
          [284.26596069, 284.53012085, 284.77182007, ...,
           283.40890503, 283.69644165, 283.98486328],
          [283.96246338, 284.20675659, 284.41360474, ...,
           283.1506958 , 283.43222046, 283.70935059],
          ...,
          [273.75601196, 273.75698853, 273.7578125 , ...,
           273.75418091, 273.75479126, 273.75524902],
          [273.69589233, 273.6960144 , 273.69616699, ...,
           273.69519043, 273.69549561, 273.69570923],
          [273.68029785, 273.68026733, 273.6802063 , ...,
           273.68048096, 273.68041992, 273.68032837]]]]])
Coordinates:
  * season     (season) <U3 'OND' 'JFM' 'AMJ' 'JAS'
  * time       (time) int64 2015 2016 2017 2018 2019 ... 2097 2098 2099 2100
  * member_id  (member_id) int64 0 1 2 3 4
  * lat        (lat) float64 64.55 65.5 66.44 67.38 ... 87.17 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    Author:    Alice DuVivier
    units:     K
    longname:  surface temperature

In [66]:
ds_to_save.to_netcdf(fout+'.nc')  # how to save file